In [47]:
import requests
import pandas as pd
from api_key import API_KEY
import time

In [2]:
v2_url = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'

count_dataset = 'pedestrian-counting-system-past-hour-counts-per-minute'
#filters = f'records?limit={10}offset={0}&timezone=UTC'

In [27]:
BASE_URL = 'https://data.melbourne.vic.gov.au/api/records/1.0/search/?dataset='
# function to get sensor location data
def sensor_location():
    sensor_dataset = 'pedestrian-counting-system-sensor-locations'
    n_records = '&q=&rows=108'
    sensor_url = f'{BASE_URL}{sensor_dataset}{n_records}'
    result = requests.get(sensor_url)
    result_json = result.json()
    records = result_json['records']
    sensor_df = pd.json_normalize(records)
    sensor_location = sensor_df[['fields.location_id', 'fields.sensor_description', 'fields.latitude', 'fields.longitude']]
    sensor_location.columns = ["Sensor ID", "Sensor Description", "lat", "lon"]
    return sensor_location

In [3]:
# Use links & iterate calls

def get_data(base, data_url, offset = 0):
    
    filters = f'records?limit={100}&offset={offset}&timezone=UTC'
    url = f'{base}{data_url}/{filters}'
    result = requests.get(url + f'&apikey={API_KEY}')
    result_json = result.json()
    max_results = result_json['total_count']
    links = result_json['links']
    records = result_json['records']
    records_df = pd.json_normalize(records)
    
    # fix columns names
    records_df.drop(columns=['links'],inplace=True)
    column_names = records_df.columns.values.tolist()
    
    column_names = [i.split('.')[-1] for i in column_names]
    records_df.columns = column_names
    
    next_url = None
    
    # get next url
    if records_df.shape[0] != max_results:
        for l in links:
            if l['rel']=='next':
                next_url = l['href']
                
    return [records_df, next_url, column_names]

In [4]:
def add_to_dataframe(dataframe, new_data):
    df = pd.concat([dataframe, new_data])
    return df

In [29]:
def fifty_minute_count(location):
    df_new, url_next, column_names = get_data(v2_url, count_dataset)
    new_ped_df = pd.DataFrame(columns=column_names)
    while url_next != None:
        df_new, url_next, column_names = get_data(v2_url, count_dataset, new_ped_df.shape[0])
        new_ped_df = add_to_dataframe(new_ped_df, df_new)
        if new_ped_df.shape[0] == 9900:
            break
    
    df.columns = ["DateTime", "Sensor ID", "Count"]
    weekdays = {
        0: "Monday",
        1: "Tuesday",
        2: "Wednesday",
        3: "Thursday",
        4: "Friday",
        5: "Saturday",
        6: "Sunday"
    }
    times = df["DateTime"]
    last_time = times.sort_values(ascending=False).iloc[1].split("T")
    weekday = time.strptime(last_time[0],"%Y-%m-%d").tm_wday
    print(f"The latest data was updated on {weekdays[weekday]} {last_time[0]} at {last_time[1].split('.')[0] } at https://data.melbourne.vic.gov.au")
    
    #Transform data
    df["Count"] = pd.to_numeric(df["Count"])
    counting = df[["Sensor ID", "Count"]].groupby(["Sensor ID"]).sum()
    counting.reset_index(level=0, inplace=True)
    location["Sensor ID"] = location["Sensor ID"].apply(lambda x: str(x))
    
    #Merge 2 dataset
    counting = pd.merge(counting, location, on='Sensor ID', how='inner')
    counting["Count"] = counting["Count"].apply(lambda x: float(x))
    return counting

NameError: name 'df' is not defined

# work space

In [32]:
def func():
    df_new, url_next, column_names = get_data(v2_url, count_dataset)
    new_ped_df = pd.DataFrame(columns=column_names)
    while url_next != None:
        df_new, url_next, column_names = get_data(v2_url, count_dataset, new_ped_df.shape[0])
        new_ped_df = add_to_dataframe(new_ped_df, df_new)
        if new_ped_df.shape[0] == 9900:
            break
    return new_ped_df

In [33]:
df = func()

In [76]:
def fifty_minute_count(location, df):
    df = df[["sensingdatetime", "locationid","total_of_directions"]]
    df.columns = ["DateTime", "Sensor ID", "Count"]
    weekdays = {
        0: "Monday",
        1: "Tuesday",
        2: "Wednesday",
        3: "Thursday",
        4: "Friday",
        5: "Saturday",
        6: "Sunday"
    }
    times = df["DateTime"]
    last_time = times.sort_values(ascending=False).iloc[1].split("T")
    weekday = time.strptime(last_time[0],"%Y-%m-%d").tm_wday
    print(f"The latest data was updated on {weekdays[weekday]} {last_time[0]} at {last_time[1].split('.')[0] } at https://data.melbourne.vic.gov.au")
    
    #Transform data
    df["Count"] = pd.to_numeric(df["Count"])
    counting = df[["Sensor ID", "Count"]].groupby(["Sensor ID"]).sum()
    counting.reset_index(level=0, inplace=True)
    
    #Merge 2 dataset
    counting = pd.merge(counting, location, on='Sensor ID', how='inner')
    counting["Count"] = counting["Count"].apply(lambda x: float(x))
    counting["Sensor ID"] = counting["Sensor ID"].apply(lambda x: str(x))
    return counting

In [78]:
location_data = sensor_location()

counting_data = fifty_minute_count(location_data, df)

counting_data.head()

The latest data was updated on Monday 2023-03-13 at 11:40:00+00:00 at https://data.melbourne.vic.gov.au


C:\Users\playroommountains\AppData\Local\Temp\ipykernel_820\252882600.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Count"] = pd.to_numeric(df["Count"])


,Sensor ID,Count,Sensor Description,lat,lon
0,1,5756.0,Bourke Street Mall (North),-37.813494,144.965153
1,2,4080.0,Bourke Street Mall (South),-37.813807,144.965167
2,3,5728.0,Melbourne Central,-37.811015,144.964295
3,4,8009.0,Town Hall (West),-37.814880,144.966088
4,5,9577.0,Princes Bridge,-37.818742,144.967877
